In [1]:
from __future__ import print_function
import cobra
import cobra.test
from os.path import join
cobra.DictList

#iRhto1108N_model with biomass as objective
iRhto1108N_model = cobra.io.read_sbml_model(join(r"C:\Users\helen\Desktop\spring2020\Rao lab\iRhto_memote-master", "iRhto.xml"))
#iRhto1108N_model with TAG as objective
iRhto1108N_model2 = cobra.io.read_sbml_model(join(r"C:\Users\helen\Desktop\spring2020\Rao lab\iRhto_memote-master", "iRhto.xml"))
iRhto1108N_model2.objective = "PDAGATpc_rm"

In [3]:
#in silico T-DNA insertion experiment of 1079 genes
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

deletion = single_gene_deletion(iRhto1108N_model)
essential = []
for i in range(len(iRhto1108N_model.genes)):
    if deletion["growth"][i] >= 2**-24:
        essential.append(deletion["growth"].axes[0][i])
# #number of NG outcome
# print(len(essential)-187)
# #number of essential genes that were not taken into account
# 1108-1079
import pandas as pd 

# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
in_vivo = pd.read_csv("knockout.csv", sep = "\t") 

# Preview the first 5 lines of the loaded data 
in_vivo.head()

G_G = 0
G_NG = 0
in_vivo_G = 0
missing = []
for h in range(len(in_vivo)):
    if in_vivo["essential"][h] == "No":
        in_vivo_G += 1
for i in range(len(essential)):
    count = 0
    for j in range(len(in_vivo)):
        if list(essential[i])[0] == in_vivo["gene"][j] and in_vivo["essential"][j] == "Yes":
            G_NG += 1
            count += 1
        if list(essential[i])[0] == in_vivo["gene"][j] and in_vivo["essential"][j] == "No":
            G_G += 1
            count += 1
    if count == 0:
        missing.append(list(essential[i])[0])
print(G_G)
print(G_NG)

737
125


In [4]:
NG_G = in_vivo_G - G_G
NG_NG = len(in_vivo) - in_vivo_G - G_NG
print(NG_G)
print(NG_NG)
accuracy = (G_G + NG_NG)/1079
sensitivity = G_G/(G_G + NG_G)
specificity = NG_NG/(NG_NG + G_NG)
print(accuracy)
print(sensitivity)
print(specificity)

82
135
0.8081556997219648
0.8998778998778999
0.5192307692307693


In [5]:
#growth yield under nitrogen limitation
from cobra.medium import minimal_medium
from cobra.flux_analysis import production_envelope
import numpy as np
import math

max_growth = iRhto1108N_model.slim_optimize()
upper_bound = minimal_medium(iRhto1108N_model, max_growth)
input_metabolites = upper_bound.index.values
STORE = np.zeros((100, 100))
with iRhto1108N_model: 
    for i in range(100):
        medium = iRhto1108N_model.medium
        medium["EX_o2_e"] = (i + 1)*12.78/100
        for j in range(100):
            medium["EX_nh4_e"] = (j + 1)*2.43/100
            for h in range(len(input_metabolites)):
                if input_metabolites[h] != "EX_nh4_e" and input_metabolites[h] != "EX_o2_e":
                    medium[input_metabolites[h]] = upper_bound[input_metabolites[h]]          
            iRhto1108N_model.medium = medium
            STORE[i][j] = iRhto1108N_model.slim_optimize()
# prod_env = production_envelope(iRhto1108N_model, ["EX_nh4_e", "EX_o2_e"], carbon_sources = "EX_glc__D_e", points = 100)
# store = np.zeros((100, 100))
# raw = np.zeros((100, 100))
# count = 0
# count1 = 0
# for i in range(9999):
#     if i != 0 and prod_env["EX_nh4_e"][i] != prod_env["EX_nh4_e"][i-1]:
#         count += 1
#         count1 = 0
#     if math.isnan(prod_env["flux_maximum"][i]):
#         raw[count1][count] = 0
#     else:
#         raw[count1][count] = prod_env["flux_maximum"][i]
#     count1 += 1

In [6]:
#growth yield under phosphate limitation
STORE1 = np.zeros((100, 100))
with iRhto1108N_model: 
    for i in range(100):
        medium = iRhto1108N_model.medium
        medium["EX_o2_e"] = (i + 1)*12.78/100
        for j in range(100):
            medium["EX_pi_e"] = (j + 1)*0.2/100
            for h in range(len(input_metabolites)):
                if input_metabolites[h] != "EX_pi_e" and input_metabolites[h] != "EX_o2_e":
                    medium[input_metabolites[h]] = upper_bound[input_metabolites[h]]          
            iRhto1108N_model.medium = medium
            STORE1[i][j] = iRhto1108N_model.slim_optimize()
# prod_env1 = production_envelope(iRhto1108N_model, ["EX_pi_e", "EX_o2_e"], carbon_sources = "EX_glc__D_e", points = 100)
# raw1 = np.zeros((100, 100))
# count = 0
# count1 = 0
# for i in range(9999):
#     if i != 0 and prod_env1["EX_pi_e"][i] != prod_env1["EX_pi_e"][i-1]:
#         count += 1
#         count1 = 0
#     if math.isnan(prod_env1["flux_maximum"][i]):
#         raw1[count1][count] = 0
#     else:
#         raw1[count1][count] = prod_env1["flux_maximum"][i]
#     count1 += 1

In [7]:
#growth yield under sulphate limitation
STORE2 = np.zeros((100, 100))
with iRhto1108N_model: 
    for i in range(100):
        medium = iRhto1108N_model.medium
        medium["EX_o2_e"] = (i + 1)*12.78/100
        for j in range(100):
            medium["EX_so4_e"] = (j + 1)*0.03/100
            for h in range(len(input_metabolites)):
                if input_metabolites[h] != "EX_so4_e" and input_metabolites[h] != "EX_o2_e":
                    medium[input_metabolites[h]] = upper_bound[input_metabolites[h]]          
            iRhto1108N_model.medium = medium
            STORE2[i][j] = iRhto1108N_model.slim_optimize()
# prod_env2 = production_envelope(iRhto1108N_model, ["EX_so4_e", "EX_o2_e"], carbon_sources = "EX_glc__D_e", points = 100)
# raw2 = np.zeros((100, 100))
# count = 0
# count1 = 0
# for i in range(9999):
#     if i != 0 and prod_env2["EX_so4_e"][i] != prod_env2["EX_so4_e"][i-1]:
#         count += 1
#         count1 = 0
#     if math.isnan(prod_env2["flux_maximum"][i]):
#         raw2[count1][count] = 0
#     else:
#         raw2[count1][count] = prod_env2["flux_maximum"][i]
#     count1 += 1

In [8]:
#TAG yield under nitrogen limitation
iRhto1108N_model2.reactions.get_by_id("Biomass_Rt_Clim").upper_bound = 0.375
max_growth2 = iRhto1108N_model2.optimize().objective_value
# upper_bound2 = minimal_medium(iRhto1108N_model2, max_growth2)
# input_metabolites2 = upper_bound2.index.values
# store3 = np.zeros((100, 100))
# with iRhto1108N_model2: 
#     for i in range(100):
#         medium = iRhto1108N_model2.medium
#         medium["EX_o2_e"] = (i + 1)*7.87/100
#         for j in range(100):
#             medium["EX_nh4_e"] = (j + 1)*1.89/100
#             for h in range(len(input_metabolites2)):
#                 if input_metabolites2[h] != "EX_nh4_e" and input_metabolites2[h] != "EX_o2_e":
#                     medium[input_metabolites2[h]] = upper_bound2[input_metabolites2[h]]          
#             iRhto1108N_model2.medium = medium
#             store3[i][j] = 1.89-iRhto1108N_model2.optimize().objective_value
store3 = np.zeros((100, 100))
prod_env3 = production_envelope(iRhto1108N_model2, ["EX_nh4_e", "EX_o2_e"], carbon_sources = "EX_glc__D_e", points = 100)
raw3 = np.zeros((100, 100))
count = 0
count1 = 0
for i in range(9999):
    if i != 0 and prod_env3["EX_nh4_e"][i] != prod_env3["EX_nh4_e"][i-1]:
        count += 1
        count1 = 0
    if math.isnan(prod_env3["flux_maximum"][i]):
        raw3[count1][count] = 0
    else:
        raw3[count1][count] = prod_env3["flux_maximum"][i]
    count1 += 1


In [ ]:
#TAG yield under phosphate limitation
store4 = np.zeros((100, 100))
prod_env4 = production_envelope(iRhto1108N_model2, ["EX_pi_e", "EX_o2_e"], carbon_sources = "EX_glc__D_e", points = 100)
raw4 = np.zeros((100, 100))
count = 0
count1 = 0
for i in range(9999):
    if i != 0 and prod_env4["EX_pi_e"][i] != prod_env4["EX_pi_e"][i-1]:
        count += 1
        count1 = 0
    if math.isnan(prod_env4["flux_maximum"][i]):
        raw4[count1][count] = 0
    else:
        raw4[count1][count] = prod_env4["flux_maximum"][i]
    count1 += 1

In [ ]:
#TAG yield under sulphate limitation
store5 = np.zeros((100, 100))
prod_env5 = production_envelope(iRhto1108N_model2, ["EX_so4_e", "EX_o2_e"], carbon_sources = "EX_glc__D_e", points = 100)
raw5 = np.zeros((100, 100))
count = 0
count1 = 0
for i in range(9999):
    if i != 0 and prod_env5["EX_so4_e"][i] != prod_env5["EX_so4_e"][i-1]:
        count += 1
        count1 = 0
    if math.isnan(prod_env5["flux_maximum"][i]):
        raw5[count1][count] = 0
    else:
        raw5[count1][count] = prod_env5["flux_maximum"][i]
    count1 += 1

In [ ]:
#plot
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

amm = np.linspace(1, 100, 100)
oxy = np.linspace(1, 100, 100)
pi1 = np.linspace(1, 100, 100)
oxy1 = np.linspace(1, 100, 100)
so42 = np.linspace(1, 100, 100)
oxy2 = np.linspace(1, 100, 100)
amm3 = np.linspace(1, 100, 50)
oxy3 = np.linspace(1, 100, 50)
pi4 = np.linspace(1, 100, 50)
oxy4 = np.linspace(1, 100, 50)
so45 = np.linspace(1, 100, 50)
oxy5 = np.linspace(1, 100, 50)

plt.rcParams['figure.dpi'] = 200
plt.rcParams['axes.labelsize'] = 6
plt.rcParams['axes.titlesize'] = 8
fig, axs = plt.subplots(3, 2)

ax = axs[0, 1]
# raw_ = raw
store = STORE
growth = store/max_growth*100
c = ax.pcolormesh(amm, oxy, growth, cmap='Reds', vmin=0, vmax=100)
ax.set_title('Growth Yield (%)')
# set the limits of the plot to the limits of the data
ax.axis([oxy.min(), oxy.max(), amm.min(), amm.max()])
ax.axis("square")
ax.set_xlabel("Ammonium uptake (%)")
ax.set_ylabel("Oxygen uptake (%)")
fig.colorbar(c, ax=ax)

ax1 = axs[1, 1]
# raw1_ = raw1
store1 = STORE1
growth1 = store1/max_growth*100
c = ax1.pcolormesh(pi1, oxy1, growth1, cmap='Reds', vmin=0, vmax=100)
ax1.set_title('Growth Yield (%)')
# set the limits of the plot to the limits of the data
ax1.axis([oxy1.min(), oxy1.max(), pi1.min(), pi1.max()])
ax1.axis("square")
ax1.set_xlabel("Phosphate uptake (%)")
ax1.set_ylabel("Oxygen uptake (%)")
fig.colorbar(c, ax=ax1)

ax2 = axs[2, 1]
# raw2_ = raw2
store2 = STORE2
growth2 = store2/max_growth*100
c = ax2.pcolormesh(so42, oxy2, growth2, cmap='Reds', vmin=0, vmax=100)
ax2.set_title('Growth Yield (%)', fontsize = 8)
# set the limits of the plot to the limits of the data
ax2.axis([oxy2.min(), oxy2.max(), so42.min(), so42.max()])
ax2.axis("square")
ax2.set_xlabel("Sulphate uptake (%)")
ax2.set_ylabel("Oxygen uptake (%)")
fig.colorbar(c, ax=ax2)

ax3 = axs[0, 0]
raw3_ = raw3[::-1, ::-1]
store3 =raw3_[5:55, 40:90]
growth3 = store3/max_growth2*100
c = ax3.pcolormesh(amm3, oxy3, growth3, cmap='Blues', vmin=0, vmax=100)
ax3.set_title('Triacylglycerol yield (%)')
# set the limits of the plot to the limits of the data
ax3.axis("square", fontsize = 8)
ax3.set_xlabel("Ammonium uptake (%)")
ax3.set_ylabel("Oxygen uptake (%)")
fig.colorbar(c, ax=ax3)

ax4 = axs[1, 0]
raw4_ = raw4[::-1, ::-1]
store4 =raw4_[5:55, 20:70]
growth4 = store4/max_growth2*100
c = ax4.pcolormesh(pi4, oxy4, growth4, cmap='Blues', vmin=0, vmax=100)
ax4.set_title('Triacylglycerol yield (%)')
# set the limits of the plot to the limits of the data
ax4.axis([oxy4.min(), oxy4.max(), pi4.min(), pi4.max()])
ax4.axis("square")
ax4.set_xlabel("Phosphate uptake (%)")
ax4.set_ylabel("Oxygen uptake (%)")
fig.colorbar(c, ax=ax4)

ax5 = axs[2, 0]
raw5_ = raw5[::-1, ::-1]
store5 =raw5_[5:55, 25:75]
growth5 = store5/max_growth2*100
c = ax5.pcolormesh(so45, oxy5, growth5, cmap='Blues', vmin=0, vmax=100)
ax5.set_title('Triacylglycerol yield (%)')
# set the limits of the plot to the limits of the data
ax5.axis([oxy5.min(), oxy5.max(), so45.min(), so45.max()])
ax5.axis("square")
ax5.set_xlabel("Sulphate uptake (%)")
ax5.set_ylabel("Oxygen uptake (%)")
fig.colorbar(c, ax=ax5)

fig.tight_layout()
plt.savefig('Phenotypic change under nutrition starvation.png')